In [1]:
from langchain_openai import AzureChatOpenAI

In [20]:
openai_api_key = "SET_IT"
openai_azure_version = "SET_IT"
openai_azure_endpoint = "SET_IT"
openai_azure_deployment = "SET_IT"

In [3]:
llm = AzureChatOpenAI(
    api_key=openai_api_key,
    api_version=openai_azure_version,
    azure_endpoint=openai_azure_endpoint,
    azure_deployment=openai_azure_deployment,
    model="gpt-4o"
)

In [4]:
llm.invoke("Who are you?")

AIMessage(content="I am an AI language model created by OpenAI. I'm here to assist you with information, answer questions, and help with various tasks you might have. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 11, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filte

In [5]:
system_prompt = """
You are an expert AI and Linguist, specializing in understanding and classifying queries based on context and intent. Below are the categories you can classify a query into. Use these categories to accurately assess the query and current chat history:

1. **general_chat**: Queries related to general topics, everyday issues, definitions, or evaluation criteria.
   
2. **provide_company_info**: Requests for specific company-related information, such as contact details (email, phone number), representative names, ownership, website, industry, or any combination of these.

3. **provide_potential_company**: Queries asking for a list of potential customers or companies related to a specific context or industry.

4. **explain_potential_reason**: Requests for an explanation of why a client or company is considered a potential candidate for a given purpose (e.g., a business partnership, investment, etc.).

5. **report_finance_insight**: Requests for financial insights, such as financial statements, consolidated reports, or detailed company performance analysis.

6. **compare_finance_insight**: Requests to compare financial data or statements at two different points in time.

7. **no_support_chat**: Queries that do not fit into any of the above categories or are unrelated to the specific tasks outlined.

### Guidelines for Classification:
- Carefully analyze the given query and historical chat context.
- If the query is ambiguous or unclear, reference prior conversation history to extract relevant information and clarify intent.
- Make the best classification decision based on the content and intent of the query, focusing on accuracy.

Start classifying the queries now!
"""

In [6]:
system_prompt2 = """
You are an expert AI and Linguist with access to the following categories:

- "provide_company_info": Used when providing company information such as email, representative name, owner, phone number, website, industry, or all of these.
- "provide_potential_company": Used when asked to provide a list of potential customers and companies.
- "explain_potential_reason": Used when explaining why a client or company is considered potential.
- "report_finance_insight": Used when providing financial statements or consolidated reports of a company.
- "compare_finance_insight": Used when comparing financial statements at two different points in time.
- "general_chat": Used for general queries about everyday issues, definitions, or evaluation criteria, even has the relevant context with the current conversation but does not fall into any of the above categories.

Your task is to classify the given query and the current chat history into one of these categories. Follow these steps:

1. **Understand the Query and Context:** Carefully read the user's query and review the chat history to gather context.
2. **Identify Relevant Information:** Look for keywords, phrases, or topics that align with the predefined categories.
3. **Classify the Query:** Based on your understanding, assign the appropriate category to the query.
4. **Handle Ambiguities:** If the query is ambiguous, refer to the chat history to extract relevant information and make an informed classification.

Always ensure that your classification is based on the information provided and stays within the predefined categories.

Begin!
"""

In [7]:
human_prompt = """
### Instructions ###
Classify given query as one of pre-defined categories. Using your ability reasoning and provided Input.
Thinking step by step

### Output Format ###
- Output must be a JSON
- key of JSON: category
- value of JSON: one of pre-defined categories
- Do not generate unncessary string i.e ```json``` in the Output and key, value must be located inside characters `""`

### Constraints ###
- Your answer should be as accurate as possible
- You should not use any external resources
- Predicted category must be stayed in pre-defined categories
- Query and History could be Vietnamese. Please leverage your language abilities to analyze carefully before determining.
- The History contains two separate roles: User and AI
    User: Query of User
    AI: Corresponding reponse of AI 
- The History is structured top-down. At the bottom of History is the latest conversation of User and AI.
- The History could be empty. You will consider this as first Query of User for this conversation.

### Examples ###

#### Example 1 ####
##### Input #####
Query: cung cấp công ty tiềm năng
History:
##### Output #####
category: provide_potential_company

#### Example 2 ####
##### Input #####
Query: Cung cấp thông tin ngành nghề  và người đại diện của công ty này
History:
User: danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
##### Output #####
category: provide_company_info

#### Example 3 ####
##### Input #####
Query: Tại sao công ty OQ1M4 HANOI And Partners là tiềm năng?
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin của công ty OQ1M4 HANOI And Partners
AI: Công ty OQ1M4 HANOI And Partners hoạt động trong lĩnh vực vật tư y tế và có các chỉ số đáng chú ý về số lượng và giá trị hóa đơn VAT cao, cho thấy công ty đang phát triển mạnh mẽ.
##### Output #####
category: explain_potential_reason

#### Example 4 ####
##### Input #####
Query: Cung cấp báo cáo tài chính của công ty OQ1M4 HANOI And Partners
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin của công ty OQ1M4 HANOI And Partners
AI: Công ty OQ1M4 HANOI And Partners có ngành nghề vật tư y tế và được đánh giá qua số lượng và giá trị hóa đơn VAT lớn, phản ánh sức mạnh tài chính của công ty.
##### Output #####
category: report_finance_insight

#### Example 5 ####
##### Input #####
Query: Cung cấp danh sách khách hàng tiềm năng trong ngành xây dựng
History:
##### Output #####
category: provide_potential_company

#### Example 6 ####
##### Input #####
Query: so sánh báo cáo tài chính công ty P5BIP CORP FAMILY 2023 và 2022
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin của công ty P5BIP CORP FAMILY
AI: Công ty P5BIP CORP FAMILY hoạt động trong ngành xây dựng và đã có những giao dịch lớn thông qua số lượng và giá trị hóa đơn VAT, cho thấy công ty đang có mức độ tiềm năng cao.
##### Output #####
category: compare_finance_insight

#### Example 7 ####
##### Input #####
Query: Cung cấp thông tin của công ty có giá trị hóa đơn cao nhất
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin của công ty có giá trị hóa đơn cao nhất
AI: Công ty có giá trị hóa đơn cao nhất là OQ1M4 HANOI And Partners trong lĩnh vực vật tư y tế.
##### Output #####
category: provide_company_info

#### Example 8 ####
##### Input #####
Query: Cung cấp thông tin ngành nghề của công ty tiềm năng nhất
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin ngành nghề của công ty tiềm năng nhất
AI: Công ty tiềm năng nhất là: 0ZUWM ORG Universal và ngành nghề kinh doanh là logistics
##### Output #####
category: provide_company_info

#### Example 9 ####
##### Input ######
Query: tiêu chí đánh giá dựa trên số lượng giá trị hóa đơn và VAT là gì?
History:
##### Output #####
category: general_chat

#### Example 10 ####
##### Input ######
Query: cách tiếp cận công ty này
History:
##### Output #####
category: general_chat

### Input ###
Query: {query}
History:
{history}

### Output ###
"""


In [ ]:
human_prompt2 = """
### INSTRUCTION ###
Classify the given query into one of the predefined categories by reasoning step by step. Break down the logic and thought process clearly before arriving at the final classification. Follow these steps:

1. **Understand the Query**: Read the user's query and identify the main request. Take note of any key details that indicate the category.
2. **Contextual Analysis**: Review the chat history to gather additional context that could help refine the classification.
3. **Chain-of-Thought**: Formulate a reasoning process by logically breaking down the query and any relevant history. This could include:
   - Identifying any entities (company names, industries, financial terms, etc.)
   - Understanding the specific nature of the request (e.g., financial data, company information, general inquiry)
4. **Determine the Category**: Using the reasoning process, classify the query into one of the predefined categories. Make sure your decision is supported by the information provided.

### FORMAT OUTPUT ###
- Return final Output directly, do not try to step by step or explain something else.
- Output must be a JSON object.
- The JSON object must contain a key "category" with the value being one of the predefined categories.
- The output must **not** contain any unnecessary strings or code blocks like ```json```.

### CONSTRAINTS ###
- Your answer should be as accurate as possible.
- Do not use any external resources; rely solely on the information in the query and the chat history.
- Make sure the predicted category is one of the predefined categories.
- The query and history may be in Vietnamese. Use your language abilities to analyze carefully before determining the category.
- The history contains two roles: **User** and **AI**:
    - **User**: The user's query.
    - **AI**: The AI's corresponding response.
- The history is structured top-down, with the latest conversation at the bottom.
- If the history is empty, treat the current query as the user's first query in the conversation.

### EXAMPLES ###

#### Example 1 ####
##### Input #####
Query: cung cấp công ty tiềm năng
History:
##### Output #####
category: provide_potential_company

#### Example 2 ####
##### Input #####
Query: Cung cấp thông tin ngành nghề  và người đại diện của công ty này
History:
User: danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
##### Output #####
category: provide_company_info

#### Example 3 ####
##### Input #####
Query: Tại sao công ty OQ1M4 HANOI And Partners là tiềm năng?
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin của công ty OQ1M4 HANOI And Partners
AI: Công ty OQ1M4 HANOI And Partners hoạt động trong lĩnh vực vật tư y tế và có các chỉ số đáng chú ý về số lượng và giá trị hóa đơn VAT cao, cho thấy công ty đang phát triển mạnh mẽ.
##### Output #####
category: explain_potential_reason

#### Example 4 ####
##### Input #####
Query: Cung cấp báo cáo tài chính của công ty OQ1M4 HANOI And Partners
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin của công ty OQ1M4 HANOI And Partners
AI: Công ty OQ1M4 HANOI And Partners có ngành nghề vật tư y tế và được đánh giá qua số lượng và giá trị hóa đơn VAT lớn, phản ánh sức mạnh tài chính của công ty.
##### Output #####
category: report_finance_insight

#### Example 5 ####
##### Input #####
Query: Cung cấp danh sách khách hàng tiềm năng trong ngành xây dựng
History:
##### Output #####
category: provide_potential_company

#### Example 6 ####
##### Input #####
Query: so sánh báo cáo tài chính công ty P5BIP CORP FAMILY 2023 và 2022
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin của công ty P5BIP CORP FAMILY
AI: Công ty P5BIP CORP FAMILY hoạt động trong ngành xây dựng và đã có những giao dịch lớn thông qua số lượng và giá trị hóa đơn VAT, cho thấy công ty đang có mức độ tiềm năng cao.
##### Output #####
category: compare_finance_insight

#### Example 7 ####
##### Input #####
Query: Cung cấp thông tin của công ty có giá trị hóa đơn cao nhất
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin của công ty có giá trị hóa đơn cao nhất
AI: Công ty có giá trị hóa đơn cao nhất là OQ1M4 HANOI And Partners trong lĩnh vực vật tư y tế.
##### Output #####
category: provide_company_info

#### Example 8 ####
##### Input #####
Query: Cung cấp thông tin ngành nghề của công ty tiềm năng nhất
History:
User: Cung cấp danh sách khách hàng tiềm năng
AI: | **Mã số thuế**  | **Tên công ty**                             | **Ngành**       | **Tiêu chí đánh giá**                          |\n|---------------|---------------------------------|--------------|--------------------------------------|\n| **9919518384**   | **0ZUWM ORG Universal**            | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **9174594029**   | **OQ1M4 HANOI And Partners**       | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **1837122591**   | **UBT8LAND PARTNERS And Partners** | logistics    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **8771910996**   | **W4C0K AND PARTNERS Universal**   | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7614514011**   | **P5BIP CORP FAMILY**              | xây dựng    | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7843088167**   | **ERT9E ORG And Partners**         | vật tư y tế  | Dựa trên số lượng và giá trị hóa đơn VAT  |\n| **7780481066**   | **1Z269 CORP National**            | logistics    | Dựa trên thông tin tài khoản  |\n| **8939286714**   | **RBSLP HCM Global**               | logistics    | Dựa trên thông tin tài khoản  |\n| **3217337923**   | **QPG4E JSC FAMILY**               | vật tư y tế  | Dựa trên thông tin tài khoản  |\n| **4053872515**   | **42PSC JSC FAMILY**               | xây dựng    | Dựa trên thông tin tài khoản  |\n\n### **Trong danh sách trên, có 10 công ty tiềm năng nhất, bao gồm:**\n- **6 công ty khai thác dựa trên nguồn hóa đơn VAT** (Các công ty có số lượng và giá trị hóa đơn VAT lớn nhất)\n- **4 công ty khai thác dựa trên thông tin tài khoản** (Các tài khoản của chủ doanh nghiệp đang hoạt động và chủ doanh nghiệp thành lập công ty mới)"
User: Cung cấp thông tin ngành nghề của công ty tiềm năng nhất
AI: Công ty tiềm năng nhất là: 0ZUWM ORG Universal và ngành nghề kinh doanh là logistics
##### Output #####
category: provide_company_info

#### Example 9 ####
##### Input ######
Query: tiêu chí đánh giá dựa trên số lượng giá trị hóa đơn và VAT là gì?
History:
##### Output #####
category: general_chat

### INPUT ###
Query: {query}
History:
{history}

### OUTPUT ###
"""

In [9]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [10]:
instruction_prompt = HumanMessagePromptTemplate.from_template(template=[{"type": "text", "text": human_prompt2}])
query_router_prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt2),
     instruction_prompt
    ]
)

In [11]:
query_router_chain = query_router_prompt | llm | JsonOutputParser()

In [12]:
out = query_router_chain.invoke({"query": "so sanh báo cáo tài chính 2023 va 2022", "history": ""})

In [13]:
out

{'category': 'compare_finance_insight'}

In [14]:
import json
import glob

In [15]:
samples = glob.glob('*.json')

In [16]:
samples

['sample1.json',
 'sample10.json',
 'sample11.json',
 'sample2.json',
 'sample3.json',
 'sample4.json',
 'sample5.json',
 'sample6.json',
 'sample7.json',
 'sample8.json',
 'sample9.json']

In [19]:
import traceback

succesful = True
for path in samples:
    with open(path, 'r') as f:
        data = json.load(f)
    
    query = data["query"]
    history = data["history"]
    history_str = ""
    for h in history:
        history_str += f"User: {h['user']}\n"
        history_str += f"AI: {h['AI']}\n"
    
    category = data["function"]
    
    c = 0
    while c < 3:
        try:
            out1 = query_router_chain.invoke({"query": query, "history": ""})
            out2 = query_router_chain.invoke({"query": query, "history": history_str})
            break
        except Exception as e:
            print("Error format", path, e)
            c += 1
            continue
    if c == 3: 
        succesful = False
    else:
        try:
            if out1["category"] != category:
                succesful = False
                print("out1", path, out1["category"], category)

            if out2["category"] != category:
                succesful = False
                print("out2", path, out2["category"], category)
        except:
            print("Error category", traceback.format_exc(path))
            
    print("================================================")
    
print("Testing: ", succesful)

Testing:  True


In [18]:
string = """
```json
{
  "category": explain_potential_reason
}
```
"""